In [4]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
from dateutil.parser import parse
from datetime import timedelta
import matplotlib.pyplot as plt

In [5]:
inputpath = './input/'
savepath= './save/'
saved_kor_csv_path = './save/Kor_csv_full/'
saved_chn_csv_path = './save/Chn_csv_full/'
korea_path = inputpath + 'from_AirKorea/'
china_path = inputpath + 'China_air/'
cities_path = inputpath + 'five_cities/'

file_ext = '.csv'

In [6]:
# functions to process the time in the data
def timeparser(time):
    return np.datetime64(parse(time))

In [7]:
# load data
'''
beijing 900
shanghai 800km
shenyang 550
'''
beijing_pm = pd.read_csv(savepath +'beijing.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
shanghai_pm = pd.read_csv(savepath +'shanghai.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
shenyang_pm = pd.read_csv(savepath +'Shenyang.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])

'''
# medile of lend
chengdu = pd.read_csv(savepath +'chengdu.csv' , parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
# south area
guangzhou = pd.read_csv(savepath +'guangzhou.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
'''
print('China data is loaded')

'''
eng_chn = pd.read_csv(savepath+ 'eng_chn.csv', parse_dates=['Year'], date_parser=timeparser, index_col=['Year'])
eng_kor = pd.read_csv(savepath+ 'eng_kor.csv', parse_dates=['Year'], date_parser=timeparser, index_col=['Year'])
print('Energy data is loaded')
'''
'''
junggu = pd.read_csv(savepath + 'junggu.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
daechi = pd.read_csv(savepath + 'daechi.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
backyoungdo = pd.read_csv(savepath + 'backyoungdo.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
gosan = pd.read_csv(savepath + 'gosan.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
suckmodo = pd.read_csv(savepath + 'suckmodo.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
duckjuckdo = pd.read_csv(savepath + 'duckjuckdo.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
print('Korea data is loaded')
'''
backyoungdo = pd.read_csv(savepath + 'backyoungdo.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])
#suckmodo = pd.read_csv(savepath + 'suckmodo.csv', parse_dates=['Date'], date_parser=timeparser, index_col=['Date'])


China data is loaded


In [8]:
#china
china = pd.DataFrame()
beijing_pm = beijing_pm.rename(index=str, columns={'Value':'Beijing'})
beijing_pm = beijing_pm.drop(['Site'], axis=1)

shanghai_pm = shanghai_pm.rename(index=str, columns={'Value':'shanghai'})
shanghai_pm = shanghai_pm.drop(['Site'], axis=1)

shenyang_pm  = shenyang_pm.rename(index=str, columns={'Value':'shenyang'})
shenyang_pm = shenyang_pm.drop(['Site'], axis=1)

#china = pd.concat([beijing, shanghai, shenyang], axis=1, keys=['Date','Date','Date'])
china = beijing_pm.merge(shanghai_pm, how='outer', left_index=True, right_index=True)
china = china.merge(shenyang_pm, how='outer', left_index=True, right_index=True)

In [9]:
def load_data():
    temp_datasets = dict()
    for file_name in os.listdir(cities_path):
        dataset_name = file_name.split('PM')[0].lower()
        dataset = pd.read_csv(cities_path+file_name,
                              parse_dates={'dt' : ['year','month','day','hour']},
                              date_parser = lambda x: pd.datetime.strptime(x, '%Y %m %d %H'),
                              infer_datetime_format=True,
                              index_col='dt',
                              na_values=['NaN','?'])
        temp_datasets[dataset_name] = dataset
        
    return temp_datasets

In [10]:
def drop_anyna(datasets):
    temp_datasets = dict()
    for city, dataset in datasets.items():
        temp_datasets[city] = dataset.dropna(axis=0, how='any')
    return temp_datasets

In [11]:
china_datasets = load_data()

In [12]:
beijing = china_datasets['beijing'].copy()

In [13]:
beijing.drop(['PM_Dongsi','PM_Dongsihuan', 'PM_Nongzhanguan', 'PM_US Post'], axis=1, inplace=True)

In [14]:
beijing.dropna(axis=0, how='any', inplace=True)

In [15]:
beijing['cbwd'] = LabelEncoder().fit_transform(beijing['cbwd'])

In [16]:
shanghai = china_datasets['shanghai'].copy()
shanghai.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52584 entries, 2010-01-01 00:00:00 to 2015-12-31 23:00:00
Data columns (total 13 columns):
No               52584 non-null int64
season           52584 non-null int64
PM_Jingan        24700 non-null float64
PM_US Post       34039 non-null float64
PM_Xuhui         25189 non-null float64
DEWP             52571 non-null float64
HUMI             52571 non-null float64
PRES             52556 non-null float64
TEMP             52571 non-null float64
cbwd             52572 non-null object
Iws              52572 non-null float64
precipitation    48575 non-null float64
Iprec            48575 non-null float64
dtypes: float64(10), int64(2), object(1)
memory usage: 5.6+ MB


In [17]:
shanghai.drop(['PM_Jingan','PM_Xuhui', 'PM_US Post'], axis=1, inplace=True)
shanghai.dropna(axis=0, how='any', inplace=True)
shanghai['cbwd'] = LabelEncoder().fit_transform(shanghai['cbwd'])
shanghai.describe()

,No,season,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec
count,48549.000000,48549.000000,48549.000000,48549.000000,48549.000000,48549.000000,48549.000000,48549.000000,48549.000000,48549.000000
mean,26250.879380,2.521576,10.858638,69.094081,1016.382263,17.127747,1.252487,51.130767,0.141189,0.918812
std,14596.417386,1.129633,9.931942,17.712016,8.986460,9.395887,1.192060,79.788204,1.063763,6.500871
min,1.000000,1.000000,-21.000000,11.320000,990.000000,-5.000000,0.000000,0.000000,0.000000,0.000000
25%,13876.000000,1.000000,3.000000,57.250000,1009.000000,9.000000,0.000000,6.000000,0.000000,0.000000
50%,26205.000000,3.000000,11.000000,71.660000,1016.900000,18.000000,1.000000,21.000000,0.000000,0.000000
75%,38415.000000,4.000000,20.000000,83.130000,1023.000000,25.000000,2.000000,63.000000,0.000000,0.000000
max,52584.000000,4.000000,28.000000,100.000000,1040.000000,41.000000,4.000000,1110.000000,61.600000,226.400000


In [18]:
shenyang = china_datasets['shenyang'].copy()
shenyang.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52584 entries, 2010-01-01 00:00:00 to 2015-12-31 23:00:00
Data columns (total 13 columns):
No               52584 non-null int64
season           52584 non-null int64
PM_Taiyuanjie    24388 non-null float64
PM_US Post       21680 non-null float64
PM_Xiaoheyan     24627 non-null float64
DEWP             51892 non-null float64
HUMI             51904 non-null float64
PRES             51892 non-null float64
TEMP             51892 non-null float64
cbwd             51892 non-null object
Iws              51892 non-null float64
precipitation    39821 non-null float64
Iprec            39821 non-null float64
dtypes: float64(10), int64(2), object(1)
memory usage: 5.6+ MB


In [19]:
shenyang.drop(['PM_Taiyuanjie','PM_Xiaoheyan', 'PM_US Post'], axis=1, inplace=True)
shenyang.dropna(axis=0, how='any', inplace=True)
shenyang['cbwd'] = LabelEncoder().fit_transform(shenyang['cbwd'])
shenyang.describe()

,No,season,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec
count,39597.000000,39597.000000,39597.000000,39597.000000,39597.000000,39597.000000,39597.000000,39597.000000,39597.000000,39597.000000
mean,30436.797308,2.386595,3.388615,63.492132,1014.589489,11.099654,2.040735,20.998076,0.083610,0.495765
std,13564.723323,1.050017,13.519116,22.484944,9.342221,12.966869,1.193575,34.154296,0.718312,4.363589
min,2156.000000,1.000000,-32.000000,7.860000,988.000000,-28.000000,0.000000,0.000000,0.000000,0.000000
25%,21217.000000,1.000000,-7.000000,46.210000,1007.000000,1.000000,1.000000,3.000000,0.000000,0.000000
50%,31315.000000,2.000000,5.000000,65.480000,1014.000000,14.000000,2.000000,9.000000,0.000000,0.000000
75%,41898.000000,3.000000,15.000000,83.010000,1021.000000,22.000000,3.000000,24.000000,0.000000,0.000000
max,52584.000000,4.000000,28.000000,100.000000,1042.000000,35.000000,4.000000,465.000000,33.100000,120.000000


In [54]:
# china = china.merge(shenyang_pm, how='outer', left_index=True, right_index=True) beijing
china_datasets = china.merge(beijing, how='outer', left_index=True, right_index=True)
china_datasets = china_datasets.merge(shanghai, how='outer', left_index=True, right_index=True)
china_datasets = china_datasets.merge(shenyang, how='outer', left_index=True, right_index=True)
china_datasets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 77342 entries, 2008-04-08 15:00:00 to 2017-06-30 23:00:00
Data columns (total 33 columns):
Beijing            75126 non-null float64
shanghai           47052 non-null float64
shenyang           33535 non-null float64
No_x               51783 non-null float64
season_x           51783 non-null float64
DEWP_x             51783 non-null float64
HUMI_x             51783 non-null float64
PRES_x             51783 non-null float64
TEMP_x             51783 non-null float64
cbwd_x             51783 non-null float64
Iws_x              51783 non-null float64
precipitation_x    51783 non-null float64
Iprec_x            51783 non-null float64
No_y               48567 non-null float64
season_y           48567 non-null float64
DEWP_y             48567 non-null float64
HUMI_y             48567 non-null float64
PRES_y             48567 non-null float64
TEMP_y             48567 non-null float64
cbwd_y             48567 non-null float64
Iws_y           

In [55]:
china_datasets['year'] = china_datasets.index.year
china_datasets['month'] = china_datasets.index.month
china_datasets['day'] = china_datasets.index.day
china_datasets['hour'] = china_datasets.index.hour
china_datasets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 77342 entries, 2008-04-08 15:00:00 to 2017-06-30 23:00:00
Data columns (total 37 columns):
Beijing            75126 non-null float64
shanghai           47052 non-null float64
shenyang           33535 non-null float64
No_x               51783 non-null float64
season_x           51783 non-null float64
DEWP_x             51783 non-null float64
HUMI_x             51783 non-null float64
PRES_x             51783 non-null float64
TEMP_x             51783 non-null float64
cbwd_x             51783 non-null float64
Iws_x              51783 non-null float64
precipitation_x    51783 non-null float64
Iprec_x            51783 non-null float64
No_y               48567 non-null float64
season_y           48567 non-null float64
DEWP_y             48567 non-null float64
HUMI_y             48567 non-null float64
PRES_y             48567 non-null float64
TEMP_y             48567 non-null float64
cbwd_y             48567 non-null float64
Iws_y           

In [ ]:
###### china data combine #####

In [56]:
def normalization(df):
    return (df-df.mean())/df.std()

In [57]:
china_datasets = normalization(china_datasets)

In [58]:
china_datasets.shape

(77342, 37)

In [20]:
### functions for AQI and UN define

In [30]:
def o3_8hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 55:
        level = 0        
    elif value < 71:
        level = 1        
    elif value < 86:
        level = 2    
    elif value < 106:
        level = 3
    else:
        level = 4        
    return level

In [31]:
def o3_1hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 125:
        level = np.NaN
    elif value < 165:
        level = 2        
    elif value < 205:
        level = 3        
    elif value < 405:
        level = 4    
    elif value < 505:
        level = 5
    else:
        level = 6        
    return level

In [32]:
def pm25_24hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 12.1:
        level = 0
    elif value < 35.5:
        level = 1        
    elif value < 55.5:
        level = 2        
    elif value < 150.5:
        level = 3    
    elif value < 250.5:
        level = 4
    elif value < 350.5:
        level = 5
    else:
        level = 6        
    return level

In [33]:
def pm10_24hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 55:
        level = 0
    elif value < 155:
        level = 1        
    elif value < 255:
        level = 2        
    elif value < 355:
        level = 3    
    elif value < 425:
        level = 4
    elif value < 505:
        level = 5
    else:
        level = 6        
    return level

In [34]:
def co_8hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 4.5:
        level = 0
    elif value < 9.5:
        level = 1        
    elif value < 12.5:
        level = 2        
    elif value < 15.5:
        level = 3    
    elif value < 30.5:
        level = 4
    elif value < 40.5:
        level = 5
    else:
        level = 6        
    return level

In [35]:
def so2_1hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 36:
        level = 0
    elif value < 76:
        level = 1        
    elif value < 186:
        level = 2
    elif value > 186:
        level = 3
    else:
        level = np.NaN        
    return level

In [36]:
def so2_24hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value >= 305 and value < 605:
        level = 4
    elif value >= 605 and value < 805:
        level = 5        
    elif value >= 805:
        level = 6        
    else:
        level = np.NaN        
    return level

In [37]:
def no2_1hr_to_aqi(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 54:
        level = 0
    elif value < 101:
        level = 1        
    elif value < 361:
        level = 2
    elif value < 650:
        level = 3
    elif value < 1250:
        level = 4
    elif value < 1650:
        level = 5
    elif value >= 1650:
        level = 6
    else:
        level = np.NaN        
    return level

In [38]:
def pm10_24hr_to_WHO(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 50:
        level = 0
    elif value >= 50:
        level = 1        
    else:
        level = np.NaN        
    return level

In [39]:
def pm10_365d_to_WHO(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 20:
        level = 0
    elif value >= 20:
        level = 1        
    else:
        level = np.NaN        
    return level

In [40]:
def pm25_24hr_to_WHO(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 25:
        level = 0
    elif value >= 25:
        level = 1        
    else:
        level = np.NaN        
    return level

In [41]:
def pm25_365d_to_WHO(value):
    if np.isnan(value):
        level = np.NaN
    elif value < 10:
        level = 0
    elif value >= 10:
        level = 1        
    else:
        level = np.NaN        
    return level

In [42]:
'''
O3 ppm -> ppb: 1000
SO2 ppm -> ppb: 1000
No2 ppm -> ppb: 1000

AQI
0: Good
1: Moderate
2: Unhealthy for sensitive
3: Unhealthy
4: Very unhealthy
5: Hazardous
6: Vary harzardous

WHO
0: good
1: bad
'''
def get_aqi_who(df):
    df_ppm = df.copy()
    '''
    O3 ppm -> ppb: 1000
    SO2 ppm -> ppb: 1000
    No2 ppm -> ppb: 1000
    '''
    df_ppm.O3 = df_ppm.O3 * 1000
    df_ppm.SO2 = df_ppm.SO2 * 1000
    df_ppm.NO2 = df_ppm.NO2 * 1000
    
    df_ppm = df_ppm.drop(['Loc', '측정소명', 'code', 'Address', '망'], axis=1)
    
    mean_8hr = df_ppm.rolling('8H').mean()
    mean_24hr = df_ppm.rolling('24H').mean()
    mean_365d = df_ppm.rolling('365D').mean()
    
    result = pd.DataFrame()
    
    aqi = pd.DataFrame()    
    aqi['8hr_o3'] = mean_8hr.O3.apply(o3_8hr_to_aqi)
    aqi['1hr_o3'] = df_ppm.O3.apply(o3_1hr_to_aqi)
    aqi['24hr_pm25'] = mean_24hr.PM25.apply(pm25_24hr_to_aqi)
    aqi['24hr_pm10'] = mean_24hr.PM10.apply(pm10_24hr_to_aqi)
    aqi['8hr_co'] = mean_8hr.CO.apply(co_8hr_to_aqi)
    aqi['1hr_so2'] = df_ppm.SO2.apply(so2_1hr_to_aqi)
    aqi['24hr_so2'] = mean_24hr.SO2.apply(so2_24hr_to_aqi)
    aqi['1hr_no2'] = df_ppm.NO2.apply(no2_1hr_to_aqi)
    aqi['aqi'] = aqi.max(axis=1)
    result['aqi'] = aqi.max(axis=1)
    
    who = pd.DataFrame()
    who['24hr_pm10_WHO'] = mean_24hr.PM10.apply(pm10_24hr_to_WHO)
    #who['365d_pm10_WHO'] = mean_365d.PM10.apply(pm10_365d_to_WHO)
    who['24hr_pm25_WHO'] = mean_24hr.PM25.apply(pm25_24hr_to_WHO)
    #who['365d_pm25_WHO'] = mean_365d.PM25.apply(pm25_365d_to_WHO)
    who['who'] = who.max(axis=1)

    result['who'] = who.max(axis=1)
    
    return aqi, who, result

In [43]:
## time shift
def time_shift(df, hours):
    df.index = df.index + timedelta(hours=hours)
    return df

In [44]:
def train_dev_test(df, train=0.9, test=0.1):
    
    train_index = int(df.shape[0] * train)
    dev_index = int(df.shape[0] * (train-test))
    
    y_train_aqi = df['aqi'].copy()[:train_index].values
    y_train_who = df['who'].copy()[:train_index].values
    x_train = df.drop(['aqi', 'who'], axis=1)
    x_train = x_train.values
    x_train = x_train[:train_index, :]
    
    y_dev_aqi = df['aqi'].copy()[dev_index:train_index].values
    y_dev_who = df['who'].copy()[dev_index:train_index].values
    x_dev = df.drop(['aqi', 'who'], axis=1)
    x_dev = x_dev.values
    x_dev = x_dev[dev_index:train_index, :]    
    
    y_test_aqi = df['aqi'].values[train_index:]
    y_test_who = df['who'].copy()[train_index:].values
    x_test = df.drop(['aqi', 'who'], axis=1)
    x_test = x_test.values
    x_test = x_test[train_index:, :]
        
    return x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who    

In [45]:
from sklearn import metrics

from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

In [46]:
def acc_precision_recall(labels, outputs):
    accuracy = metrics.accuracy_score(labels, outputs)
    f1_score = metrics.f1_score(labels, outputs)
    #precision = metrics.average_precision_score(labels, outputs)
    
    recall = metrics.recall_score(labels, outputs)
    auc = metrics.roc_auc_score(labels, outputs)

    print('In the test, Accuracy: {:.4f}, Precision: {:.4f}, recall: {:.4f}, AUC: {:.4f}'.format(accuracy, precision, recall, auc))

In [47]:
def models_test(X_train, Y_train, X_test, Y_test):
    # stochastic gradient descent (SGD) learning
    sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
    sgd.fit(X_train, Y_train)
    Y_pred = sgd.predict(X_test)
    f1_sgd = metrics.f1_score(Y_test, Y_pred, average='weighted')
    
    # Random Forest
    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(X_train, Y_train)
    Y_pred = random_forest.predict(X_test)
    f1_random_forest = metrics.f1_score(Y_test, Y_pred, average='weighted')
    
    # Logistic Regression
    logreg = LogisticRegression()
    logreg.fit(X_train, Y_train)
    Y_pred = logreg.predict(X_test)
    f1_log = metrics.f1_score(Y_test, Y_pred, average='weighted')

    # KNN
    knn = KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    f1_knn = metrics.f1_score(Y_test, Y_pred, average='weighted')

    # Gaussian Naive Bayes
    gaussian = GaussianNB()
    gaussian.fit(X_train, Y_train)
    Y_pred = gaussian.predict(X_test)
    f1_gaussian = metrics.f1_score(Y_test, Y_pred, average='weighted')

    # Perceptron
    perceptron = Perceptron(max_iter=5)
    perceptron.fit(X_train, Y_train)
    Y_pred = perceptron.predict(X_test)
    f1_perceptron = metrics.f1_score(Y_test, Y_pred, average='weighted')

    # Linear SVC
    linear_svc = LinearSVC()
    linear_svc.fit(X_train, Y_train)
    Y_pred = linear_svc.predict(X_test)
    f1_linear_svc = metrics.f1_score(Y_test, Y_pred, average='weighted')

    # Decision Tree
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(X_train, Y_train)
    Y_pred = decision_tree.predict(X_test)
    f1_decision_tree = metrics.f1_score(Y_test, Y_pred, average='weighted')

    results = pd.DataFrame({
        'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                  'Random Forest', 'Naive Bayes', 'Perceptron', 
                  'Stochastic Gradient Decent', 
                  'Decision Tree'],
        'Score': [f1_linear_svc, f1_knn, f1_log, 
                  f1_random_forest, f1_gaussian, f1_perceptron, 
                  f1_sgd, f1_decision_tree]})
    result_df = results.sort_values(by='Score', ascending=False)
    result_df = result_df.set_index('Score')
    result_df.head(9)
    results = results.set_index('Model')
    return results

In [48]:
def time_shift_and_model_test(china, location, start, finsh):
    time_shifting = start
    
    result = pd.DataFrame()
    aqi, who, level = get_aqi_who(location)
    level = time_shift(level, time_shifting - 1)
    
    while time_shifting < finsh:
        
        print('time shift: {}'.format(time_shifting))
        # shifting an hour        
        level = time_shift(level, 1)
        
        # merge x and y
        data_location = china.merge(level, how='outer', left_index=True, right_index=True)
        data_location = data_location.dropna()

        #print(data_location.notna().sum())        
        
        
        # make train and test
        x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who = train_dev_test(data_location)
        
        # do the test in the model with WHO
        test_result = models_test(x_train, y_train_who, x_test, y_test_who)
        
        # rename the result and concat it
        tag = 'who ts_' + str(time_shifting)
        test_result = test_result.rename(index=str, columns={'Score':tag})
        
        result = pd.concat([test_result, result], axis=1)
        
        # do the test in the model with AQI
        test_result = models_test(x_train, y_train_aqi, x_test, y_test_aqi)
        
        # rename the result and concat it
        tag = 'aqi ts_' + str(time_shifting)
        test_result = test_result.rename(index=str, columns={'Score':tag})
        
        result = pd.concat([test_result, result], axis=1)
        time_shifting += 1
    return result

In [60]:
result = time_shift_and_model_test(china_datasets, location=backyoungdo, start=2, finsh=25)

time shift: 2


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 3


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 4


...

C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 23


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 24


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [61]:
pd.set_option('display.max_columns', 100)
result

,aqi ts_24,who ts_24,aqi ts_23,who ts_23,aqi ts_22,who ts_22,aqi ts_21,who ts_21,aqi ts_20,who ts_20,aqi ts_19,who ts_19,aqi ts_18,who ts_18,aqi ts_17,who ts_17,aqi ts_16,who ts_16,aqi ts_15,who ts_15,aqi ts_14,who ts_14,aqi ts_13,who ts_13,aqi ts_12,who ts_12,aqi ts_11,who ts_11,aqi ts_10,who ts_10,aqi ts_9,who ts_9,aqi ts_8,who ts_8,aqi ts_7,who ts_7,aqi ts_6,who ts_6,aqi ts_5,who ts_5,aqi ts_4,who ts_4,aqi ts_3,who ts_3,aqi ts_2,who ts_2
Model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Support Vector Machines,0.647903,0.695597,0.638732,0.697683,0.629330,0.700845,0.644668,0.702427,0.645805,0.702427,0.649759,0.700561,0.656087,0.699770,0.659218,0.701352,0.654465,0.700561,0.652131,0.702651,0.651326,0.705310,0.645701,0.705310,0.644436,0.704518,0.640238,0.703443,0.648849,0.701068,0.655474,0.699487,0.658520,0.700277,0.658935,0.698696,0.655179,0.702991,0.658639,0.704099,0.663451,0.706226,0.659133,0.710185,0.662462,0.711964
KNN,0.591202,0.703498,0.578056,0.706594,0.580151,0.708741,0.589726,0.705826,0.592201,0.704615,0.586382,0.706172,0.575679,0.703751,0.572414,0.695964,0.576667,0.691054,0.573941,0.672842,0.571588,0.675910,0.570110,0.679432,0.569864,0.671578,0.563614,0.670758,0.571354,0.669291,0.578595,0.663927,0.576888,0.658769,0.574637,0.654957,0.566982,0.659600,0.571896,0.662067,0.579456,0.663203,0.580821,0.674211,0.581178,0.683325
Logistic Regression,0.650229,0.694464,0.638028,0.696264,0.632688,0.699425,0.645564,0.701007,0.653749,0.701007,0.656997,0.699425,0.656847,0.698350,0.655586,0.699932,0.652650,0.699141,0.649067,0.701799,0.652515,0.704174,0.648733,0.704174,0.646925,0.703666,0.639451,0.702875,0.644352,0.700501,0.652907,0.698919,0.657224,0.699994,0.661496,0.698413,0.656264,0.699994,0.659971,0.701410,0.660155,0.703548,0.656755,0.706158,0.659811,0.710633
Random Forest,0.569393,0.730073,0.576595,0.752337,0.590639,0.736922,0.614934,0.751894,0.559069,0.772142,0.557030,0.734567,0.612899,0.717426,0.558226,0.746077,0.593088,0.736542,0.610651,0.732663,0.599302,0.735745,0.623999,0.716178,0.622973,0.739829,0.643271,0.719188,0.654813,0.745555,0.636226,0.784456,0.635973,0.754438,0.664446,0.698332,0.662504,0.734898,0.659661,0.750271,0.683800,0.754964,0.672200,0.704539,0.692235,0.736234
Naive Bayes,0.408119,0.547209,0.414444,0.551717,0.420043,0.549475,0.422292,0.552230,0.415037,0.561872,0.423195,0.552334,0.432495,0.549608,0.442335,0.547196,0.445394,0.546145,0.413793,0.543022,0.421225,0.544620,0.435306,0.545690,0.437380,0.550412,0.435388,0.556347,0.425109,0.562038,0.393432,0.568132,0.389587,0.562938,0.384788,0.567710,0.372501,0.569270,0.350806,0.569554,0.336056,0.566853,0.323392,0.561299,0.306626,0.549039
Perceptron,0.457633,0.695881,0.470423,0.680494,0.547505,0.704414,0.569835,0.733830,0.519793,0.661520,0.394960,0.701071,0.589713,0.647391,0.361088,0.699435,0.594841,0.729491,0.260124,0.705904,0.583259,0.747190,0.566867,0.709677,0.468074,0.675626,0.415938,0.690562,0.523686,0.594768,0.582884,0.561159,0.611741,0.735980,0.480485,0.688780,0.565609,0.723459,0.466718,0.526658,0.561074,0.638026,0.543721,0.710545,0.633686,0.681818
Stochastic Gradient Decent,0.578002,0.695314,0.537592,0.702465,0.592990,0.677533,0.542500,0.702968,0.607152,0.711429,0.272750,0.739917,0.559150,0.716726,0.324501,0.734617,0.610139,0.699994,0.535234,0.694632,0.612124,0.707198,0.445919,0.698083,0.614734,0.705085,0.622453,0.710940,0.598853,0.692473,0.627172,0.713203,0.632418,0.651388,0.591439,0.717964,0.554101,0.668145,0.673463,0.694422,0.560160,0.702331,0.496099,0.592775,0.537712,0.706670
Decision Tree,0.516699,0.642092,0.629771,0.651560,0.621522,0.662021,0.508110,0.565979,0.592747,0.571441,0.410784,0.639898,0.412937,0.652241,0.539700,0.642759,0.567517,0.703382,0.490176,0.708863,0.340991,0.685985,0.453686,0.685977,0.503329,0.681369,0.438726,0.707923,0.497638,0.691650,0.483676,0.715425,0.443774,0.669701,0.563008,0.710771,0.547561,0.712054,0.542729,0.754046,0.577280,0.697429,0.474424,0.693452,0.591463,0.686749


In [62]:
from sklearn.decomposition import PCA

In [63]:
def time_shift_and_model_test_w_pca(china, location, start, finsh, pca=0.99):
    time_shifting = start
    
    result = pd.DataFrame()
    aqi, who, level = get_aqi_who(location)
    level = time_shift(level, time_shifting - 1)
    
    while time_shifting < finsh:
        
        print('time shift: {}'.format(time_shifting))
        # shifting an hour        
        level = time_shift(level, 1)
        
        # merge x and y
        data_location = china.merge(level, how='outer', left_index=True, right_index=True)
        data_location = data_location.dropna()
         
        # make train and test
        x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who = train_dev_test(data_location)
        
        # pca
        pca = PCA(.99).fit(x_train)
        k = pca.n_components_
        print('PCA components: {}'.format(k))
        pca = PCA(n_components=k)

        pca.fit(x_train)
        x_train = pca.transform(x_train)
        x_test = pca.transform(x_test)
        
        # do the test in the model with WHO
        test_result = models_test(x_train, y_train_who, x_test, y_test_who)
        
        # rename the result and concat it
        tag = 'who ts_' + str(time_shifting)
        test_result = test_result.rename(index=str, columns={'Score':tag})
        
        result = pd.concat([test_result, result], axis=1)
        
        # do the test in the model with AQI
        test_result = models_test(x_train, y_train_aqi, x_test, y_test_aqi)
        
        # rename the result and concat it
        tag = 'aqi ts_' + str(time_shifting)
        test_result = test_result.rename(index=str, columns={'Score':tag})
        
        result = pd.concat([test_result, result], axis=1)
        time_shifting += 1
    return result

In [64]:
result = time_shift_and_model_test_w_pca(china_datasets, location=backyoungdo, start=2, finsh=25)

time shift: 2
PCA components: 25


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 3
PCA components: 25


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 4
PCA components: 25


...

C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 23
PCA components: 25


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time shift: 24
PCA components: 25


C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\woare\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [66]:
pd.set_option('display.max_columns', 100)
result

,aqi ts_24,who ts_24,aqi ts_23,who ts_23,aqi ts_22,who ts_22,aqi ts_21,who ts_21,aqi ts_20,who ts_20,aqi ts_19,who ts_19,aqi ts_18,who ts_18,aqi ts_17,who ts_17,aqi ts_16,who ts_16,aqi ts_15,who ts_15,aqi ts_14,who ts_14,aqi ts_13,who ts_13,aqi ts_12,who ts_12,aqi ts_11,who ts_11,aqi ts_10,who ts_10,aqi ts_9,who ts_9,aqi ts_8,who ts_8,aqi ts_7,who ts_7,aqi ts_6,who ts_6,aqi ts_5,who ts_5,aqi ts_4,who ts_4,aqi ts_3,who ts_3,aqi ts_2,who ts_2
Model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Support Vector Machines,0.629827,0.695597,0.628973,0.697966,0.628389,0.701128,0.629511,0.702710,0.641676,0.702710,0.644324,0.700845,0.645272,0.700054,0.652255,0.701636,0.657498,0.700845,0.651876,0.702935,0.651629,0.705594,0.650688,0.705310,0.653580,0.704518,0.654600,0.703443,0.660667,0.701068,0.663743,0.699487,0.662620,0.700277,0.660417,0.698696,0.656442,0.700277,0.660155,0.699769,0.662076,0.700560,0.661194,0.704516,0.658014,0.704966
KNN,0.590387,0.701863,0.576142,0.706170,0.582560,0.708313,0.591233,0.707027,0.594010,0.705005,0.584664,0.705718,0.573321,0.701699,0.568952,0.695849,0.575823,0.691349,0.573109,0.675228,0.569841,0.677483,0.568990,0.677092,0.572033,0.669678,0.568017,0.669663,0.570765,0.669738,0.576099,0.664745,0.575195,0.658387,0.577163,0.654502,0.573931,0.661135,0.574597,0.665974,0.583536,0.666733,0.585159,0.675141,0.580520,0.685140
Logistic Regression,0.628383,0.695314,0.626841,0.697400,0.629813,0.700277,0.632392,0.701860,0.644913,0.701860,0.644441,0.700277,0.647327,0.699487,0.650089,0.701068,0.653786,0.700277,0.650559,0.702651,0.654543,0.705026,0.649627,0.705026,0.653883,0.704234,0.651936,0.703443,0.658683,0.701068,0.662219,0.699487,0.665969,0.700277,0.665308,0.698413,0.659788,0.699994,0.661233,0.698413,0.659434,0.700560,0.656653,0.704516,0.655656,0.705251
Random Forest,0.613086,0.698581,0.630451,0.694226,0.613860,0.703331,0.629116,0.702442,0.613325,0.698060,0.616221,0.699053,0.631479,0.692354,0.631968,0.706270,0.634093,0.700547,0.647078,0.714645,0.628744,0.703624,0.615861,0.706630,0.630457,0.691278,0.633433,0.700078,0.619550,0.699408,0.630776,0.702718,0.645257,0.708536,0.641387,0.704949,0.644338,0.709175,0.651714,0.699459,0.657820,0.705669,0.662506,0.705882,0.642896,0.704965
Naive Bayes,0.551501,0.689198,0.548516,0.688157,0.486444,0.553125,0.465056,0.571176,0.434678,0.609057,0.431109,0.641162,0.455778,0.668347,0.473022,0.680629,0.494667,0.679651,0.500328,0.674048,0.502421,0.674740,0.523892,0.667215,0.528085,0.662059,0.527724,0.643151,0.494893,0.591339,0.460104,0.580186,0.441740,0.615107,0.405731,0.619103,0.367891,0.601733,0.330014,0.557266,0.316019,0.547900,0.317918,0.538101,0.305459,0.511637
Perceptron,0.548224,0.682334,0.559756,0.710479,0.532113,0.688828,0.547511,0.702143,0.536089,0.692621,0.518886,0.623614,0.558700,0.675537,0.493501,0.687049,0.535026,0.732256,0.495641,0.698782,0.568972,0.756787,0.580941,0.738407,0.530297,0.621743,0.600231,0.670948,0.618023,0.603127,0.607288,0.668692,0.569884,0.695573,0.547825,0.707702,0.647041,0.673680,0.561105,0.682639,0.554537,0.655587,0.649283,0.698215,0.650088,0.714091
Stochastic Gradient Decent,0.500246,0.695246,0.380667,0.701739,0.557956,0.707427,0.412911,0.699583,0.631692,0.699476,0.596795,0.700845,0.659707,0.738929,0.518690,0.734228,0.504813,0.691111,0.558168,0.721521,0.640273,0.728364,0.602636,0.704458,0.648666,0.763543,0.575638,0.707680,0.553955,0.675858,0.544139,0.696672,0.597473,0.755847,0.510094,0.694302,0.380386,0.696001,0.619864,0.699377,0.671359,0.702490,0.624257,0.691128,0.621995,0.684517
Decision Tree,0.529266,0.659662,0.503289,0.683185,0.557960,0.671863,0.494754,0.651779,0.517143,0.682668,0.541525,0.664862,0.539520,0.687085,0.525541,0.662596,0.549110,0.692640,0.536567,0.728943,0.543738,0.678983,0.557279,0.701503,0.570016,0.707340,0.522198,0.709866,0.518145,0.659055,0.574100,0.698848,0.542981,0.677569,0.566545,0.676241,0.577373,0.697331,0.557341,0.662043,0.549836,0.693003,0.624846,0.688033,0.576584,0.697271


In [67]:
def time_shift_w_pca_dataset(china, location, timeshift, pca=0.99):
    
    aqi, who, level = get_aqi_who(location)
    level = time_shift(level, timeshift)
    
    print('time shift: {}'.format(timeshift))
    # shifting an hour        
    level = time_shift(level, 1)

    # merge x and y
    data_location = china.merge(level, how='outer', left_index=True, right_index=True)
    data_location = data_location.dropna()

    # make train and test
    x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who = train_dev_test(data_location)

    # pca
    pca = PCA(.99).fit(x_train)
    k = pca.n_components_
    print('PCA components: {}'.format(k))
    pca = PCA(n_components=k)

    pca.fit(x_train)
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)

    return x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who

In [68]:
x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who = time_shift_w_pca_dataset(china_datasets, backyoungdo, 6)

time shift: 6
PCA components: 25


In [69]:
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [70]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [71]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train_who)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 33.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 70.0min finished


{'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [79]:
def acc_precision_recall_auc_f1(labels, outputs):
    accuracy = metrics.accuracy_score(labels, outputs)
    precision = metrics.precision_score(labels, outputs, average='weighted')
    recall = metrics.recall_score(labels, outputs, average='weighted')
    auc = metrics.roc_auc_score(labels, outputs, average='weighted')
    f1_score = metrics.f1_score(labels, outputs, average='weighted')
    
    print('In the test, Accuracy: {:.4f}, Precision: {:.4f}, recall: {:.4f}, AUC: {:.4f}, f1: {:.4f}'.format(accuracy, precision, recall, auc, f1_score))
    
    return accuracy, precision, recall, auc, f1_score

In [80]:
def evaluate(model, test_features, test_labels):
    predictions = np.round(model.predict(test_features))
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    accuracy, precision, recall, auc, f1_score = acc_precision_recall_auc_f1(test_labels, predictions)
    
    
    
    return predictions

In [81]:
base_model = RandomForestClassifier(n_estimators = 100, random_state = 40)
base_model.fit(x_train, y_train_who)
base_predictions = evaluate(base_model, x_test, y_test_who)

Model Performance
Average Error: 0.2245 degrees.
Accuracy = nan%.
In the test, Accuracy: 0.7755, Precision: 0.6922, recall: 0.7755, AUC: 0.5112, f1: 0.7104


C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [82]:
best_random = rf_random.best_estimator_
best_predictions = evaluate(best_random, x_test, y_test_who)

Model Performance
Average Error: 0.2153 degrees.
Accuracy = nan%.
In the test, Accuracy: 0.7847, Precision: 0.6610, recall: 0.7847, AUC: 0.4987, f1: 0.6992


C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [84]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test_who, best_predictions).ravel()
(tn, fp, fn, tp)

(1364, 15, 360, 3)

In [85]:
def time_shift_dataset(china, location, timeshift):
    
    aqi, who, level = get_aqi_who(location)
    level = time_shift(level, timeshift)
    
    print('time shift: {}'.format(timeshift))
    # shifting an hour        
    level = time_shift(level, 1)

    # merge x and y
    data_location = china.merge(level, how='outer', left_index=True, right_index=True)
    data_location = data_location.dropna()

    # make train and test
    x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who = train_dev_test(data_location)

    return x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who

In [86]:
x_train, y_train_aqi, y_train_who, x_test, y_test_aqi, y_test_who = time_shift_dataset(china_datasets, backyoungdo, 6)

time shift: 6


In [87]:
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [88]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [89]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train_who)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 28.7min finished


{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': False}

In [90]:
base_model = RandomForestClassifier(n_estimators = 100, random_state = 40)
base_model.fit(x_train, y_train_who)
base_predictions = evaluate(base_model, x_test, y_test_who)

Model Performance
Average Error: 0.2067 degrees.
Accuracy = nan%.
In the test, Accuracy: 0.7933, Precision: 0.7601, recall: 0.7933, AUC: 0.5945, f1: 0.7652


C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [91]:
best_random = rf_random.best_estimator_
best_predictions = evaluate(best_random, x_test, y_test_who)

Model Performance
Average Error: 0.1969 degrees.
Accuracy = nan%.
In the test, Accuracy: 0.8031, Precision: 0.7794, recall: 0.8031, AUC: 0.5468, f1: 0.7416


C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\woare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [92]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test_who, best_predictions).ravel()
(tn, fp, fn, tp)

(1360, 19, 324, 39)